In [1]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

Warning message:
: package ‘foreach’ was built under R version 3.2.5Loading required package: iterators
Warning message:
: package ‘iterators’ was built under R version 3.2.5Loading required package: parallel
Warning message:
: package ‘Matching’ was built under R version 3.2.3Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

Warning message:
: package ‘plyr’ was built under R version 3.2.5Warning message:
: package ‘tidyverse’ was built under R version 3.2.5Warning message:
: replacing previous import by ‘tidyr::%>%’ when loading ‘broom’Warning message:
: replacing previous import by ‘tidyr::gather’ when loading ‘broom’Warning message:
: 

In [2]:
loop_test = function(setup) {
    DGP = setup$DGP
    n_train = setup$n_train
    n_test = setup$n_test
    n_folds = setup$n_folds
    training_percent = setup$training_percent
    rep = setup$rep
    models = setup$models
    
    n = n_train + n_test
    training_percent 
    
    datas = setup_data(DGP$DGP, n_train, n_test, n_folds)
    estimates = datas %$% get_estimates(data, models, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_train=n_train, n_test=n_test, n_folds=n_folds, rep=rep)
    
    return(true_selection_error)
}

In [3]:
methods = list("ranger",
               "enet")
tune_grids = list(expand.grid(mtry = c(3,5,7),
                              splitrule="variance",
                              min.node.size=1),
                  expand.grid(fraction=0.5, 
                              lambda=exp(seq(-5,2,0.5))))
models = list(methods, tune_grids) %>%
    pmap(~list(method=.x, tune_grid=.y))

simulation_params = list(DGP = powers_DGPs() %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n_test = 2000,
                         n_train = 1000,
                         n_folds = c(2,5,10),
                         rep = 2:) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [4]:
# results = setups %>% 
#     map(~loop_test(.)) %>%
#     bind_rows

In [5]:
cl = makeCluster(4)
registerDoParallel(cl)
 
results = foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

stopCluster(cl)

In [6]:
write_csv(results, "data/experimental_results.csv")